### Import Libraries and Data Preprocessing

In [ ]:
# Load required libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
import pandas as pd
import tkinter as tk
from tkinter import ttk, messagebox
import pickle

# Ignore only FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [49]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import pickle

# Load the pre-trained XGBoost model
with open("HP_AAC_xgb_avoa_model.pkl", "rb") as file:
    final_model = pickle.load(file)

# Get the feature names (all columns except the target column)
features = ['Fiber aspect ratio (%)', 'Fiber volume fraction (%)', 'Water (kg)',
            'Water/Binder', 'Solution/Binder ', 'Flow (mm)', 'Fly ash (FA)',
            'Silica fume (SF)', 'Slag (GGBS)', 'Sodium Hydroxide (NaOH) (SH)',
            'Sodium Silicate (SS) (waterglass) Na2SiO3', 'SS/SH Ratio',
            'NaOH Conc. (Molarity)', 'Quartz Powder (%)', 'Quartz Sand (%)',
            'Sodium metasilicate Na2SiO3(M) (NS)', 'Curing Temp (degree)', 'Curing Time (Days)']

# Create tkinter window
root = tk.Tk()
root.title("Predict Compressive Strength of HP AAC")

# Set dark mode colors
background_color = "#BDBEC9"
label_color = "#030303"
entry_color = "#4b4b4b"
button_color = "#3c78d8"
text_color = "#ffffff"
result_color = "#030303"

root.configure(bg=background_color)

# Main frame to hold the scrollable content
main_frame = tk.Frame(root, bg=background_color)
main_frame.pack(fill=tk.BOTH, expand=1)

# Create a canvas in the main frame
canvas = tk.Canvas(main_frame, bg=background_color)
canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=1)

# Add a scrollbar to the canvas
scrollbar = ttk.Scrollbar(main_frame, orient=tk.VERTICAL, command=canvas.yview)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

# Configure canvas with scrollbar
canvas.configure(yscrollcommand=scrollbar.set)
canvas.bind('<Configure>', lambda e: canvas.configure(scrollregion=canvas.bbox("all")))

# Create another frame inside the canvas
input_frame = tk.Frame(canvas, bg=background_color)
canvas.create_window((0, 0), window=input_frame, anchor="nw")

# Styling for the input fields
entry_style = {
    'font': ('Roboto', 12),
    'bg': entry_color,
    'fg': text_color,
    'relief': tk.GROOVE,
    'insertbackground': text_color  # Cursor color
}

label_style = {
    'font': ('Roboto', 12, 'bold', 'italic'),
    'bg': background_color,
    'fg': label_color,
    'anchor': 'w'
}

# Add a bold label for "Input Features"
title_label = tk.Label(input_frame, text="Input Features", font=('Roboto', 14, 'bold', 'italic'), bg=background_color, fg=label_color)
title_label.grid(row=0, column=0, columnspan=2, pady=10)

# Create a dictionary to hold input entries
entries = {}

# Add input fields for each feature dynamically
for i, feature in enumerate(features):
    label = tk.Label(input_frame, text=f"{feature}:", **label_style)
    label.grid(row=i+1, column=0, sticky="w", padx=10, pady=5)
    
    entry = tk.Entry(input_frame, **entry_style)
    entry.grid(row=i+1, column=1, padx=10, pady=5)
    
    entries[feature] = entry

# Function to handle the prediction
def predict_compressive_strength():
    input_data = []
    
    # Gather data from input fields
    for feature, entry in entries.items():
        value = entry.get().strip()
        if value:
            try:
                input_data.append(float(value))  # Convert input to float
            except ValueError:
                messagebox.showerror("Error", f"Invalid input for {feature}. Please enter a valid number.")
                return
        else:
            messagebox.showerror("Error", f"Please fill in all fields for {feature}.")
            return

    # Convert to DataFrame for the model
    input_df = pd.DataFrame([input_data], columns=features)
    
    # Predict using the loaded model
    prediction = final_model.predict(input_df)
    
    # Display prediction result
    result_label.config(text=f"Predicted Compressive Strength: {prediction[0]:.2f} MPa")

# Add the Predict button at the end
predict_button = tk.Button(input_frame, text="Predict", command=predict_compressive_strength, bg=button_color, fg=label_color, font=('Roboto', 12, 'italic'), padx=10, pady=5)
predict_button.grid(row=len(features)+1, column=1, pady=20)

# Add a label to display the prediction result
result_label = tk.Label(input_frame, text="", font=('Roboto', 14, 'bold', 'italic'), bg=background_color, fg=result_color)
result_label.grid(row=len(features)+2, column=0, columnspan=2, pady=20)

# Set a minimum window size
root.geometry("600x500")

# Run the application
root.mainloop()
